# Ollama on Colab - qwen2.5-coder-32b

# =============================================
# IMPORTANT: How to get your ngrok authtoken
# =============================================
# 1. Go to https://ngrok.com and sign up for a free account
# 2. After signing in, go to https://dashboard.ngrok.com/get-started/your-authtoken
# 3. Copy your authtoken
# 4. In Google Colab:
#    - Click on the key icon in the left sidebar to open "Secrets"
#    - Click "Add new secret"
#    - Set "Name" as: authtoken
#    - Set "Value" as: your-ngrok-token-here
#    - Click "Add"
# 5. Run this notebook with a GPU runtime (Runtime -> Change runtime type -> GPU)
# =============================================

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import userdata
userdata.get('authtoken')

'2hRFFRiub4do3XOPSTIj5RHc232_2MNnfhnysxZsVEHmdGddp'

Here is the code that will:

- Install Ollama
- Setup ngrok to acces your Ollama server from outside of Colab
- Download the **qwen2.5-coder-32b** LLM model.
- Be patient, it's a 20GB model!

In [ ]:
!curl https://ollama.ai/install.sh | sh

!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers

!pip install pyngrok
from pyngrok import ngrok
ngrok.set_auth_token(userdata.get('authtoken'))

import os
import asyncio

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )
from IPython.display import clear_output
clear_output()

await asyncio.gather(
run_process(['ollama', 'serve']),
run_process(['ollama', 'pull', 'qwen2.5-coder:32b']),
run_process(['ngrok', 'http', '--log', 'stderr', '11434', '--host-header="localhost:11434"'])
)

>>> starting ollama serve
>>> starting ollama pull qwen2.5-coder:32b
>>> starting ngrok http --log stderr 11434 --host-header="localhost:11434"
2024/11/20 01:11:53 routes.go:1189: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://*] OLLAMA_SCHED_SPRE

Enjoy!